# Image Comparison with Image Quality Assessment (IQA)

This notebook can be used to compare two images / volumes using different IQA metrics. The metrics used are:
- Peak Signal to Noise Ratio (PSNR)
- Root Mean Square Error (RMSE)
- Structured Similarity (SSIM)
- Multi-Scale Structural Similarity (MS-SSIM) [can only be used for 2D images]
- Feature Similarity Index (FSIM) [can only be used for 2D images]
- Visual Information Fidelity in **pixel** domain (VIFp) [can only be used for 2D images]

The calculated values for VIFp, SSIM, MS-SSIM are probably not correct in this implementation. Those values should be treated with caution.

The metrics are implemented to calculate the scores for a 8 bit data range (0-255) per default. The data range can be changed by setting the `data_range` variable for each metric. Images are first loaded from .raw files or .mhd files and their corresponding .raw file, normalized to the chosen data range and then compared. The scores are then calculated and printed. The images are plotted to visually compare them. File names need to be given with the bit depth denoted as a suffix (e.g. _8bit.raw, _16bit.mhd) and the dimensions of the images need to be given in the file name (e.g. _512x512x512_). The images are assumed to be grayscale.

To use this notebook, first set the filepaths and parameters, the press 'Run All'.

The following packages have to be installed:
- numpy
- matplotlib
- pytorch
- piq
- scikit-image

## Import

In [2]:
import psnr, rmse, fsim, vif, ssim, msssim
from utils import load_data
import matplotlib.pyplot as plt

## Data Loading and Setup

In [3]:
# Change these filepaths to your images (_r = reference, _m = modified)
file_path_img_r = '../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_1620proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'
file_path_img_m = '../samples/Catec_Two_PlateIQI_20um/Catec_Two_PlateIQI_20um_810proj_220kV_Rayscan-SimCT_800x800x1000_16bit.raw'

# Set these to true if you want to calculate the denominated metric for your data
calc_psnr = True
calc_rmse = True
calc_ssim = True
calc_fsim = False
calc_vif = False

In [4]:
# Data loading
img_r = load_data(file_path_img_r)
img_m = load_data(file_path_img_m)

## Calculation

### PSNR
Set the parameters for PSNR. The data range is set to 255 per default.

In [5]:
# Change this parameter
psnr_data_range = 255

In [6]:
if calc_psnr:
    metric_psnr_m = psnr.PSNR(data_range=psnr_data_range)
    metric_psnr_m.score(img_r, img_m)

### RMSE
The RMSE metric does not need any parameters.

In [7]:
if calc_rmse:
    metric_rmse_m = rmse.RMSE(data_range=255)
    metric_rmse_m.score(img_r, img_m)

### SSIM
Set the parameters for SSIM. The data range is set to 255 per default.

In [8]:
ssim_data_range = 255

In [9]:
if calc_ssim:
    metric_ssim_m = ssim.SSIM(data_range=ssim_data_range)
    metric_ssim_m.score(img_r, img_m)

MemoryError: Unable to allocate 4.77 GiB for an array with shape (1000, 800, 800) and data type float64

### FSIM
Set the parameters for FSIM. The data range is set to 255 per default. The chromatic parameter can be set to true if the images are RGB images.

In [10]:
# Change these parameters
fsim_data_range = 255
fsim_parameters = {'chromatic': False}

In [11]:
if calc_fsim:
    if len(img_r.shape) == 2:
        metric_fsim_m = fsim.FSIM(fsim_parameters, data_range=fsim_data_range)
        metric_fsim_m.score(img_r, img_m)
    else:
        print('FSIM can only be used for 2D images')

### VIFp
Set the parameters for VIFp. The data range is set to 255 per default. The sigma_n_sq parameter can be set to adapt the metric for the variance of visual noise. This parameter is set to 3.2 per default and has to be changed according to the data. Currently there is no recommendation how to set this parameter.

In [12]:
# Change these parameters
vif_data_range = 255
vif_parameters = {'sigma_n_sq': 3.2}

In [13]:
if calc_vif:
    if len(img_r.shape) == 2:
        metric_vifp_m = vif.VIFp(vif_parameters, data_range=255)
        metric_vifp_m.score(img_r, img_m)
    else:
        raise Exception('VIFp can only be used for 2D images')

In [14]:
# Set the number of decimals to be printed
decimals = 2

In [15]:
if calc_psnr:
    metric_psnr_m.print_score(decimals)
if calc_rmse:
    metric_rmse_m.print_score(decimals)
if calc_ssim:
    metric_ssim_m.print_score(decimals)
if calc_fsim:
    metric_fsim_m.print_score(decimals)
if calc_vif:
    metric_vifp_m.print_score(decimals)

PSNR: 29.86
RMSE: 8.19
No score value for SSIM. Run score() first.


## Plotting

In [ ]:
print('Shape of reference image:', img_r.shape)

In [ ]:
# Set the area to be plotted (two dimensions have to be given as range, the third value is the slice to be plotted)
x = None
x_1 = 0
x_2 = 1000

y = 200
y_1 = None
y_2 = None

z = None
z_1 = 0
z_2 = 800

In [ ]:
if x is not None:
    img_r_plot = img_r[x, y_1:y_2, z_1:z_2]
    img_m_plot = img_m[x, y_1:y_2, z_1:z_2]
elif y is not None:
    img_r_plot = img_r[x_1:x_2, y, z_1:z_2]
    img_m_plot = img_m[x_1:x_2, y, z_1:z_2]
elif z is not None:
    img_r_plot = img_r[x_1:x_2, y_1:y_2, z]
    img_m_plot = img_m[x_1:x_2, y_1:y_2, z]
else:
    raise Exception('Area to be plotted was not correctly specified')

In [ ]:
fig, axs = plt.subplots(1,2, dpi=300)
axs[0].imshow(img_r_plot, cmap="gray")
axs[1].imshow(img_m_plot, cmap="gray")

fig.suptitle('Image Comparison and IQA metric values', y=0.92)
axs[0].set_title('Reference image')
axs[1].set_title('Modified image')

if calc_psnr:
    fig.text(0.5, 0.12, f'RMSE: {metric_rmse_m.score_val:.2f}', ha='center', fontsize=8)
if calc_rmse:
    fig.text(0.5, 0.09, f'PSNR: {metric_psnr_m.score_val:.2f} dB', ha='center', fontsize=8)
if calc_ssim:
    fig.text(0.5, 0.06, f'SSIM: {metric_ssim_m.score_val:.2f}', ha='center', fontsize=8)
if calc_fsim:
    fig.text(0.5, 0.03, f'FSIM: {metric_fsim_m.score_val:.2f}', ha='center', fontsize=8)
if calc_vif:
    fig.text(0.5, 0.00, f'VIFp: {metric_vifp_m.score_val:.2f}', ha='center', fontsize=8)
    
axs[0].axis('off')
axs[1].axis('off');